# 交易策略：搜尋過去10個交易日裡的強勢&弱勢股
##買進標準：
>* 跳空開高2%以上的天數 > 1
>* 長紅k3%以上的天數 > 2
>* 今日高比前一天的高還高&今日低比前一天的低點還高 的天數 > 5
>* 次日只要再漲3%就會創近10日新高
>* 近3日均量超過近10日均量的20%
---
##賣出標準：
>* 跳水跌2%以上的天數 1
>* 黑k3%以上的天數 > 2
>* 今日高沒過前一天的高 且 今日低卻跌破前一天的低 的天數 > 5
>* 次日只要再跌超過3%就會創近10日新低
>* 黑k的天數中、上影線超過1%的天數 > 1


In [1]:
!pip install -q  yfinance # 用 colab執行時，才需每次運行時都安裝此模組
import yfinance as yf
import pandas as pd
from tqdm import tqdm
def ohlc(ticker, days = -10):
    return pd.DataFrame(yf.Ticker(ticker).history()[days:])[['Open','High','Low','Close','Volume']]

# 筆者自製的標普500成份股
sid1 = pd.read_csv('https://raw.githubusercontent.com/MaxChenCMC/SnP500_candlestick_pattern_filter/master/S%26P500.csv')['Symbol'].to_list() 
# yahoo finance的screener多種主題選股
sid2 = pd.read_html('https://finance.yahoo.com/screener/predefined/day_gainers?count=100&offset=0')[0].Symbol.to_list()
sid3 = pd.read_html('https://finance.yahoo.com/screener/predefined/small_cap_gainers?count=100&offset=0')[0].Symbol.to_list() 
sid4 = pd.read_html('https://finance.yahoo.com/screener/predefined/most_actives?count=100&offset=0')[0].Symbol.to_list() 
sid5 = pd.read_html('https://finance.yahoo.com/screener/predefined/growth_technology_stocks?count=100&offset=0')[0].Symbol.to_list() 
sid6 = pd.read_html('https://finance.yahoo.com/screener/predefined/undervalued_growth_stocks?count=100&offset=0')[0].Symbol.to_list() 
sid7 = pd.read_html('https://finance.yahoo.com/screener/predefined/undervalued_large_caps?count=100&offset=0')[0].Symbol.to_list()
sid8 = pd.read_html('https://finance.yahoo.com/screener/predefined/aggressive_small_caps?count=100&offset=0')[0].Symbol.to_list()
# 把以上撈出的股號做成 watchlist觀察清單
watchlist = set(sid1 + sid2 + sid3 + sid4 + sid5 + sid6 + sid7 + sid8)

# 選股條件的係數，若篩出股票太多就把係數調高(變嚴格)
x1, x2, x3, x4, x5 = 1, 2, 5, 0.03, 1.2

long = []
short = []
for i in tqdm(watchlist):
    res = ohlc(str(i),-10)
    try:
        if ( (len([res['Open'] > res['Close'].shift()*1.02]) >= x1) 
        & (len(res[res['Close'] > res['Open']*1.03]) >= x2) 
        & (len( res[(res['High'] > res['High'].shift()) & ( res['Low'] > res['Low'].shift())] ) >= x3) 
        & (res['Close'][-1]*(1+x4) >= res['High'][-10:].max()) 
        & (res['Volume'][-3:].mean() > res['Volume'][-10:].mean()*x5) ):
            long.append(i) 
        elif ( (len(res[res['Open'] < res['Close'].shift()*0.98]) >= x1 )
        &(len(res[res['Close']*1.03 < res['Open']]) >= x2 )
        &(len(res[ (res['High'] < res['High'].shift()) &(res['Low'] < res['Low'].shift()) ]) >= x3)
        &(res['Close'][-1]*(1-x4) <= res['Low'][-10:].min())
        &(len( res[res['Close'] < res['Open']] [res['High'] > res['Open']*1.01] ) >= 1) ):
            short.append(i) 
        else: pass
    except Exception as e:
        print('公司',i,'報價有誤，或許近日曾被暫停交易' )
print('---'*20)
print(f'做多{long}\n做空{short}')

 23%|██▎       | 216/929 [00:25<01:23,  8.53it/s]

- RTN: No data found for this date range, symbol may be delisted
公司 RTN 報價有誤，或許近日曾被暫停交易


100%|██████████| 929/929 [01:49<00:00,  8.48it/s]

------------------------------------------------------------
做多['ABR', 'WORK', 'OCFT', 'RNG', 'SONO', 'AXL', 'RCII', 'PYPL', 'PRMW', 'MRNA', 'PRTS', 'SQ', 'TWLO', 'UI', 'LNC', 'DLPH', 'FLIR']
做空['ALXN', 'DLR']


# 在finviz開啟
把網址後綴的股號字串替換成本次篩選出來的即可

In [0]:
import webbrowser
webbrowser.open_new('https://www.finviz.com/screener.ashx?v=211&t=' + long) # colab無法開啟網頁，需手動貼上網址

![img](https://i.imgur.com/O9mAKYq.png)